# Project Title

## Overview

Our company is making the venture into the airline industry. We have been tasked with determining which aircraft carry the lowest risk. In this notebook, we have provided three recommendation for our company's airplane purchases.

## Business Understanding

stakeholder and business questions

## Data Understanding

Data source and properties and why relevant to problem

The data has been provided in this [Aviation Accident Database 1948-2022](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses) which is publicly available through [kaggle](https://www.kaggle.com/) datasets.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../Final_Data/Aviation_Data.csv', parse_dates=['Event.Date', 'Publication.Date'])
df.head()

explore properties of data and why it's well suited to solivng the prob

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.describe(datetime_is_numeric=True)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

Exploring columns that we may want to drop

In [ ]:
df['Air.carrier'].value_counts()

In [ ]:
df['Purpose.of.flight'].value_counts()

In [ ]:
df['FAR.Description'].value_counts()

In [ ]:
df['Schedule'].value_counts()

In [ ]:
df['Report.Status'].value_counts()

## Data Preparation

rationalize every step

Making a subset. I.e. dropping columns we don't want from the data understanding

Explaining which columns we chose to drop b/c obviously not relevant or redundant: event.id, ivestigation.type, accident.number. latitude, longitude, registration.number, report.status, publication.date

Explaining columns which might be relevant to other business questions: airport.code, airport.name, FAR Description, schedule, purpose.of.flight, air.carrier, broad.phase.of.flight

### Making a Subset by Dropping Columns and Rows

In [ ]:
df_subset = df[['Event.Date', 'Location', 'Country', 'Number.of.Engines',
                'Aircraft.damage', 'Aircraft.Category', 'Make', 'Model', 
                'Amateur.Built', 'Engine.Type',
                'Total.Fatal.Injuries', 'Total.Minor.Injuries', 
                'Total.Uninjured', 'Total.Serious.Injuries', 
                'Weather.Condition']]


In [ ]:
# Standardizing the formatting of the column names

df_subset = df_subset.rename(columns = lambda x: x.lower())

In [ ]:
# Keeping only airplanes

df_subset = df_subset.loc[df_subset['aircraft.category'] == 'Airplane']

In [ ]:
# Dropping the 'aircraft.category' column

df_subset = df_subset.drop(columns=['aircraft.category'])

In [ ]:
# Keeping only non-amateur built airplanes

df_subset = df_subset.loc[df_subset['amateur.built'] == 'No']

In [ ]:
# Dropping the 'amateur.built' column

df_subset = df_subset.drop(columns=['amateur.built'])

In [ ]:
# Keeping only the year (first 4 characters) from the 'event.date' column

df_subset = df_subset.rename(columns = lambda x: x.lower())
df_subset['event.date'] = pd.to_datetime(df_subset['event.date']).dt.year


In [ ]:
# Keeping only the state abbreviations from 'location' column into a new column

df_subset['state'] = df_subset['location'].str.split(",").str[1]


In [ ]:
# Dropping the 'location' column

df_subset = df_subset.drop(columns=['location'])

In [ ]:
# Exploring the 'weather.condition' column

df_subset['weather.condtion'].value_counts()

In [ ]:
# Cleaning typos from weather.condition

replace_dict = {'Unk':'UNK'}
df_subset['weather.condition'] = df_subset['weather.condition'].replace(replace_dict)
df_subset['weather.condition'].value_counts()

In [ ]:
# Exploring the 'engine.type' column

df_subset['engine.type'].value_counts()

In [ ]:
# Cleaning typos from engine.type

replace_dict3 = {'UNK':'Unknown'}
df_subset['engine.type'] = df_subset['engine.type'].replace(replace_dict3)
df_subset['engine.type'].value_counts()

Understanding for our new data set having dropped unecessary info and cleaned redundant values

In [ ]:
df_subset.head()

In [ ]:
df_subset.shape

In [ ]:
df_subset.describe()

In [ ]:
df_subset.info()

### Cleaning Null Values

In [ ]:
df_subset.isna().sum()

In [ ]:
# Dropping rows in columns that have very few nulls

df_subset = df_subset.dropna(subset=['country', 'make', 'model', 'state'])
df_subset.isna().sum()

In [ ]:
# Filling the NaN's in the 4 injuries columns with 0's

df_subset['total.serious.injuries'].fillna(0, inplace=True)
df_subset['total.fatal.injuries'].fillna(0, inplace=True)
df_subset['total.minor.injuries'].fillna(0, inplace=True)
df_subset['total.uninjured'].fillna(0, inplace=True)

df_subset.isna().sum()

In [ ]:
# Replacing nulls in the 'aircraft.damage' column

df_subset['aircraft.damage'].value_counts()
df_subset['aircraft.damage'] = df_subset['aircraft.damage'].fillna('N/A')

In [ ]:
# Combining 'Unknown' and 'N/A' values in in the 'aircraft.damage' column

replace_dict2 = {'Unknown':'N/A'}
df_subset['aircraft.damage'] = df_subset['aircraft.damage'].replace(replace_dict2)
df_subset['aircraft.damage'].value_counts()

In [ ]:
df_subset2.isna().sum()

In [ ]:
# Replacing nulls in the 'engine.type' column

df_subset['engine.type'].value_counts()
df_subset['engine.type'] = df_subset['engine.type'].fillna('Unknown')

In [ ]:
df_subset.isna().sum()

In [ ]:
# Replacing nulls in the 'weather.condition' column

df_subset['weather.condition'].value_counts()
df_subset['weather.condition'] = df_subset['weather.condition'].fillna('N/A')

In [ ]:
df_subset.isna().sum()

Now, to deal with nulls in the 'number.of.engines' column, we will search for an appropriate measure of central tendency. We cannot fill the nulls with 0's, as this doesn't make conceptual sense: there are no planes with no engines.

In [ ]:
# Checking the distribution of the 'number.of.engines' column

df_subset['number.of.engines'].value_counts()

In [ ]:
number.engines.hist, ax = plt.subplots
ax.hist(number.of.engines, bins = 7)
ax.set_xlabel('number.of.engines')
ax.set_ylabel('count')
ax.set_title('Distribution of number.of.engines')

In [ ]:
# Replacing nulls in the 'number.of.engines' column

df_subset['number.of.engines'].median()
df_subset['number.of.engines'] = df_subset['number.of.engines'].fillna(df_subset['number.of.engines'].median())

### Consolidating the serious injuries and minor injuries columns

why we do this

In [ ]:
df_subset2['total.nonfatal.injuries'] = df_subset['total.minor.injuries'] + df_subset['total.serious.injuries']

In [ ]:
df_subset = df_subset.drop(columns=['total.serious.injuries', 'total.minor.injuries'])

### Typecasting: Changing float columns into integer columns 

Why we do this

In [ ]:
df_subset.dtypes

In [ ]:
df_subset['number.of.engines'] = df_subset['number.of.engines'].astype(int)

In [ ]:
df_subset['total.serious.injuries'] = df_subset['total.serious.injuries'].astype(int)
df_subset['total.fatal.injuries'] = df_subset['total.fatal.injuries'].astype(int)
df_subset['total.minor.injuries'] = df_subset['total.minor.injuries'].astype(int)
df_subset['total.uninjured'] = df_subset['total.uninjured'].astype(int)

In [ ]:
df_subset.dtypes

### Calculating/creating new injuries percentage columns

In [ ]:
# Turning injury columns into percentage of total passengers
df_subset['fatal.injuries.perc'] = round(((df_subset['total.fatal.injuries'] / (df_subset2['total.fatal.injuries'] + 
                                                                             df_subset2['total.nonfatal.injuries'] + 
                                                                             df_subset2['total.uninjured'])) * 100), 1)
df_subset['nonfatal.injuries.perc'] = round(((df_subset['total.nonfatal.injuries'] / (df_subset2['total.fatal.injuries'] + 
                                                                             df_subset2['total.nonfatal.injuries'] + 
                                                                             df_subset2['total.uninjured'])) * 100), 1)
df_subset['uninjured.perc'] = round(((df_subset['total.uninjured'] / (df_subset2['total.fatal.injuries'] + 
                                                                             df_subset2['total.nonfatal.injuries'] + 
                                                                             df_subset2['total.uninjured'])) * 100), 1)

In [ ]:
# Cleaning resulting nulls

df_subset.isna().sum()

df_subset['fatal.injuries.perc'].fillna(0, inplace=True)
df_subset['nonfatal.injuries.perc'].fillna(0, inplace=True)
df_subset['uninjured.perc'].fillna(0, inplace=True)

In [ ]:
df_subset.isna().sum()

In [ ]:
# Turning the new injuries percentage columns into integers

df_subset['fatal.injuries.perc'] = df_subset['fatal.injuries.perc'].astype(int)
df_subset['nonfatal.injuries.perc'] = df_subset['nonfatal.injuries.perc'].astype(int)
df_subset['uninjured.perc'] = df_subset['uninjured.perc'].astype(int)

df_subset.info()

### Translating the aircraft damage column into a numerical scale

why we do this

In [ ]:
df_subset['aircraft.damage'].value_counts()

In [ ]:
def aircraft_damage_numbers(y):
    if y == "Substantial":
        return 3
    elif y == "Destroyed":
        return 2
    elif y == "Minor":
        return 1
    else:
        return 0

In [ ]:
df_subset['aircraft.damage.scale'] = df_subset['aircraft.damage'].map(aircraft_damage_numbers)
df_subset['aircraft.damage.scale'].value_counts()

### Cleaning the 'make' column

In [ ]:
df_subset['make'].head()

In [ ]:
# Standardizing the capitalization of every value

df_subset['make'] = df_subset['make'].astype(str).str.lower()
pd.set_option('display.max_rows', None)
df_subset['make'].value_counts().head(50)

In [ ]:
# Defining a function to fix the typos in the most common makes

def typos(x):
    if "air tractor" in x:
        return "air tractor"
    elif "cessna" in x:
        return "cessna"
    elif "piper" in x:
        return "piper"
    elif "beech" in x:
        return "beech"
    elif "boeing" in x:
        return "boeing"
    elif "mooney" in x:
        return "mooney"
    elif "grumman" in x:
        return "grumman"
    elif "airbus" in x:
        return "airbus"
    elif "aeronca" in x:
        return "aeronca"
    elif "cirrus" in x:
        return "cirrus"
    elif "champion" in x:
        return "american champion"
    elif "embraer" in x:
        return "embraer"
    elif "havilland" in x:
        return "dehavilland"
    elif "aviat" in x:
        return "aviat"
    elif "diamond" in x:
        return "diamond"
    elif "ercoupe" in x:
        return "ercoupe"
    else:
        return x

In [ ]:
df_subset['make'] = df_subset['make'].map(typos)

### Defining and saving our cleaned subset

We are dropping columns used for calculations of percentages and aircraft damage scale

In [ ]:
df_subset = df_subset[['event.date', 'country', 'state', 'number.of.engines',
                'make', 'model', 'engine.type',  
                 'fatal.injuries.perc', 'nonfatal.injuries.perc', 'uninjured.perc', 
                'weather.condition', 'aircraft.damage.scale']]

In [ ]:
df_subset.head()

In [ ]:
df_subset.shape

In [ ]:
df_subset.info()

In [ ]:
df_subset.describe()

In [ ]:
df_subset.to_csv('../Final_Data/subset.csv')

why we aren't re-indexing the subset

### Defining a second subset that only contains plane makes (brands) with over 100 entries

In [ ]:
make_value_counts = df_subset['make'].value_counts()

In [ ]:
make_over_100 = make_value_counts.loc[make_value_counts > 100]

In [ ]:
make_over_100 = list(make_over_100.index)

In [ ]:
make_over_100

In [ ]:
df_subset_makes = df_subset.loc[df_subset['make'].isin(make_over_100)]
df_subset_makes.head()

In [ ]:
df_subset_makes.to_csv('../Final_Data/df_subset_makes.csv')

In [ ]:
df_subset2_makes.info()

In [ ]:
df_subset2_makes.describe()

Conclusion: we now have two data subsets. Explain them

# Analysis and Results

the groupby's and the visuals produced in tableau embedded following those cells

In [ ]:
df_subset.groupby('event.date').mean()

In [ ]:
df_subset.groupby('country').mean()

In [ ]:
df_subset.groupby('state').mean()

In [ ]:
df_subset.groupby('number.of.engines').mean()

In [ ]:
df_subset_makes.groupby('make').mean()

In [ ]:
df_subset.groupby('engine.type').mean()

In [ ]:
df_subset.groupby('weather.condition').mean()

In [ ]:
df_subset.groupby('aircraft.damage.scale').mean()

### Business Recommendation 1

In [ ]:
df_subset.groupby('engine.type')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

### Business Recommendation 2

In [ ]:
df_subset.groupby('number.of.engines')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

### Business Recommendation 3

In [ ]:
df_subset_makes.groupby('make')['aircraft.damage.scale', 'fatal.injuries.perc'].mean()

## Conclusion

Airbus or Boeing Turbo Fan Planes
Chartered flights: single engine planes
Commercial flights: dual engine planes


### Next Steps